# SPARQL Queries for Data Sync between Crest and a Knowledge Graph

The following SPARQL queries must be set to allow data sync between Crest Annotation and the Digital Heraldry Knowledge Graph.

Variables that are passed from outside the SPARQL query are set as `{{variable}}` inside the respective query.

It is assumed, that the following variables can be passed between Crest and the SPARQL queries:
- `manifestIRI`: URL of the IIIF manifest of which the current image is part of
- `iiifImageURL`: URL of the current image file as it is given in the IIIF manifest of which the current image is part of
- `croppedImageLink`: URL, adhearing to the IIIF image API standard, that displays onl the contents of an image annotation

Note on terminology: In general, everything that pertains to the image files is organised in and represented as a IIIF manifest and is addressed as such in the following notebook. Everything addressedd as a _conceptual_ representation of something (e.g. a IIIF canvas) refers to something that is stored as an entity in the Knowledge Graph, which does not directly represent a digital image but can be linked to one. For example: A page from a book is represented conceptually in the KG as a `dhoo:ManuscriptFolio` (and may have according metadata; e.g. a specific date, material properties, …). This conceptual representation is linked to an image (=the digital scan of the page). This image file of a page from a book has a URL and is modelled as a _canvas_ inside a IIIF manifest.

## Export Data from Crest into the KG

Prerequisites in the KG:
- A conceptual representation of the image collection/IIIF manifest that is exported from Crest must already exist in the KG (e.g. as `dhoo:Manuscript` or another subclass of `dhoo:Object`). This entails, that the `dhoo:Manuscript` also has already in the KG been linked to its IIIF manifest.

Check if the current IIIF manifest already exists in the KG

In [ ]:
PREFIX dhor: <http://digitalheraldry.org/dho/representation#>
PREFIX dhoo: <http://digitalheraldry.org/dho/object#>
PREFIX dhoe: <http://digitalheraldry.org/dho/entity#>

ASK {
  ?manuscript a dhoo:Manuscript ;
              dhoo:hasIIIFResource <{{manifestIRI}}> .
}

Check if the current IIIF image file with a linked conceptual representation of the page/folio already exists in the KG

In [ ]:
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

PREFIX dhor: <http://digitalheraldry.org/dho/representation#>
PREFIX dhoo: <http://digitalheraldry.org/dho/object#>
PREFIX dhoe: <http://digitalheraldry.org/dho/entity#>

PREFIX iiif: <http://iiif.io/api/presentation/2#>
PREFIX oa: <http://www.w3.org/ns/oa#>

ASK {
  ?manuscript a dhoo:Manuscript ;
              dhoo:hasIIIFResource <{{manifestIRI}}> .

  <{{manifestIRI}}> iiif:hasSequences ?sequence .
  ?sequence rdf:rest*/rdf:first ?firstSequence .
  ?firstSequence iiif:hasCanvases ?canvasList .
  ?canvasList rdf:rest*/rdf:first ?canvas .
  ?canvas rdfs:label ?canvasLabel .
  ?canvas iiif:hasImageAnnotations ?annotations .
  ?annotations rdf:rest*/rdf:first ?anno .
  ?anno oa:hasBody <{{iiifImageURL}}> .
  
  ?manuscriptFolioIRI dhoo:hasImageFile <{{iiifImageURL}}> .
}

### Case 1: A representation of the image does not yet exists in the KG

In this case, the representation of the image in the KG (as `dhoo:ManuscriptFolio`) has to be added as well as the annotations, that were created in Crest.
The following branch has to be followed if the previous SPARQL ASK query returned `false`. (Otherwise, continue with _Case 2_.)

Adding the IIIF manifest into the KG is quicker by uploading the manifests' json file directly into the KGs' triplestore

In [ ]:
curl -X POST -H "Content-Type: application/ld+json" --data-binary @{iiif_manifest_filename} {sparql_endpoint_url}

Takes an IRI `{{manifestIRI}}` of a IIIF manifest and creates a dhoo:ManuscriptFolio entity for each canvas that is in the manifest. Each dhoo:ManuscriptFolio is automatically connected to the dhoo:Object in the KG that uses the manifest IRI with dhoo:hasIIIFResource.

In [ ]:
# create-object-parts-from-manifest.sparql

PREFIX dhoo: <http://digitalheraldry.org/dho/object#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX iiif: <http://iiif.io/api/presentation/2#>
PREFIX oa: <http://www.w3.org/ns/oa#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

INSERT {
    ?newObjectPartURI dhoo:partOfObject ?object .
    ?newObjectPartURI a dhoo:ManuscriptFolio .
    ?newObjectPartURI a owl:NamedIndividual .
    ?newObjectPartURI dhoo:folioNumber ?canvasLabel .
    ?newObjectPartURI dhoo:hasImageFile ?imageURL .
}
WHERE {
    BIND( <{{manifestIRI}}> AS ?manifestIRI ) . 
    ?object dhoo:hasIIIFResource ?manifestIRI .
    ?manifestIRI iiif:hasSequences ?sequence .
    ?sequence rdf:rest*/rdf:first ?firstSequence .
    ?firstSequence iiif:hasCanvases ?canvasList .
    ?canvasList rdf:rest*/rdf:first ?canvas .
    ?canvas rdfs:label ?canvasLabel .
    ?canvas iiif:hasImageAnnotations ?annotations .
    ?annotations rdf:rest*/rdf:first ?anno .
    ?anno oa:hasBody ?imageURL .

    # Create new URI for each object part
    BIND( IRI(CONCAT( str(?object), "-", REPLACE(?canvasLabel, " ", "%20") )) AS ?newObjectPartURI )
}

Continue with _Case 2_

### Case 2: A representation of the image already exists in the KG

In this case, only the annotations, that were created in Crest, have to be added to the KG. The following SPARQL queries ensure, that they are linked to the correct `dhoo:ManuscriptFolio` (whose image representaion is the current one in Crest).

Return the URI of the conceptual representation of the current IIIF image file. The query returns the URI as `manuscriptFolioIRI`, `canvasLabel`, as well as `manuscriptIRI` (which represents the conceptual representation of the whole IIIF manifest in the KG).

In [ ]:
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

PREFIX dhor: <http://digitalheraldry.org/dho/representation#>
PREFIX dhoo: <http://digitalheraldry.org/dho/object#>
PREFIX dhoe: <http://digitalheraldry.org/dho/entity#>

PREFIX iiif: <http://iiif.io/api/presentation/2#>
PREFIX oa: <http://www.w3.org/ns/oa#>

SELECT ?manuscriptFolioIRI ?manuscriptIRI ?canvasLabel
WHERE {
  ?manuscriptIRI a dhoo:Manuscript ;
           dhoo:hasIIIFResource <{{manifestIRI}}> .

  <{{manifestIRI}}> iiif:hasSequences ?sequence .
  ?sequence rdf:rest*/rdf:first ?firstSequence .
  ?firstSequence iiif:hasCanvases ?canvasList .
  ?canvasList rdf:rest*/rdf:first ?canvas .
  ?canvas rdfs:label ?canvasLabel .
  ?canvas iiif:hasImageAnnotations ?annotations .
  ?annotations rdf:rest*/rdf:first ?anno .
  ?anno oa:hasBody <{{iiifImageURL}}> .
  
  ?manuscriptFolioIRI dhoo:hasImageFile <{{iiifImageURL}}> .
}

**QUESTION @Dominik:** Haben wir in Crest eine im gesamten Crest-Projekt/IIIF-Manifest eindeutige numerische ID für jede Annotation? Falls das der Fall ist, kann die uuid in der `representationIRI` im Folgenden durch eben diese numerische ID ersetzt werden. Falls es eine solche ID nicht gibt, kann alles so belassen werden.

The `representationIRI` has to be constructed outside of the SPARQL query with the following schema:

In [ ]:
representationIRI = manuscriptIRI.replace('http://digitalheraldry.org/dho/object#', 'http://digitalheraldry.org/dho/representation#')
representationIRI = representationIRI + '-' + uuid4()

Create a conceptual representation of a annotation and link it to the image annotation from Crest

In [ ]:
# create-representations-from-bbox-coordinates.sparql

PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX dhor: <http://digitalheraldry.org/dho/representation#>
PREFIX dhoo: <http://digitalheraldry.org/dho/object#>

INSERT {
    <{{representationIRI}}> a dhor:CoatOfArmsVisualRepresentation ;
        a dhor:CoatOfArmsRepresentation ;
        a owl:NamedIndividual ;
        dhor:folioNumber "{{canvasLabel}}" ;
        dhoo:locatedOnFolio <{{manuscriptFolioIRI}}> ;
        dhor:locatedOnObject <{{manuscriptIRI}}> ;
        dhoo:hasImageFile <{{croppedImageLink}}> .
}
WHERE {
}

## Import Data from the KG into Crest

It is assumed, that in Crest a project with a IIIF manifest has already been created. This IIIF manifest is used to import the correct data from the KG.

Load all annotations for a given `manifestIRI`.

In [ ]:
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

PREFIX dhor: <http://digitalheraldry.org/dho/representation#>
PREFIX dhoo: <http://digitalheraldry.org/dho/object#>
PREFIX dhoe: <http://digitalheraldry.org/dho/entity#>

PREFIX iiif: <http://iiif.io/api/presentation/2#>
PREFIX oa: <http://www.w3.org/ns/oa#>
PREFIX dhoh: <http://digitalheraldry.org/dho/heraldry#>

SELECT ?folioImageFileURL ?blazon ?blazonTextAnnotation ?annotationImageFile
WHERE {
  ?manuscriptIRI a dhoo:Manuscript ;
           dhoo:hasIIIFResource <{{manifestIRI}}> .
  
  ?manuscriptFolio dhoo:partOfObject ?manuscriptIRI ;
                   dhoo:hasImageFile ?folioImageFileURL .
  
  # Load annotations
  OPTIONAL {
    ?coaRepresentation dhoo:locatedOnFolio ?manuscriptFolio ;
                       dhoo:hasImageFile ?annotationImageFile .
    
    # Load KG entity that describes the annotation
    OPTIONAL {
      ?coaRepresentation dhor:hasBlazoningAct ?blazonAct .
      ?blazonAct dhor:hasBlazon ?blazon .
      
      # Load textual transcription
      OPTIONAL {
        ?blazon dhoh:hasBlazonFromOMA ?blazonTextAnnotation .
      }
    }
  }
}

The coordinates of each annotation can be extracted from the IIIF URL of the respective `annotationImageFile`:

https://gallica.bnf.fr/iiif/ark:/12148/btv1b55009806h/f155/**7,1100,656,764**/full/0/native.jpg